In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from sentinelhub import SentinelHubRequest, DataCollection, MimeType, CRS, BBox, SHConfig
from dotenv import load_dotenv
from pyproj import Transformer
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import folium
import cv2

In [14]:
load_dotenv()  # charge automatiquement le .env du dossier courant

True

In [15]:
list_bbox_centers = [(43.52960344286241, 5.448962145567533),
(48.092094070852475, -1.7036876101196967),
 (48.121222106840435, -1.602864906604979),
 (45.74557103359813, 4.803091960652229),
 (47.8776770361181, -3.909947991759771),
 (44.55378060775165, 4.28389205759154),
 (45.69764498502806, 5.8945655967385315),
 (48.07500134978729, 7.3491231648955155),
 (50.63412206750312, 3.0435779303880106),
 (44.83078182200822, 0.5108123933396876)]

In [16]:
from urban_watch.ml_logic.data import RAW_DATA_DIR
print(RAW_DATA_DIR)

/home/k2805/code/augustinlafond/project_urban_watch/raw_data


In [17]:
from urban_watch.ml_logic.data import make_bbox_global, get_data

data = get_data(list_bbox_centers)

📡 Downloading tile 0 at 43.52960344286241,5.448962145567533 ...
✔ Saved tile 0 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_0
📡 Downloading tile 1 at 48.092094070852475,-1.7036876101196967 ...
✔ Saved tile 1 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_1
📡 Downloading tile 2 at 48.121222106840435,-1.602864906604979 ...
✔ Saved tile 2 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_2
📡 Downloading tile 3 at 45.74557103359813,4.803091960652229 ...
✔ Saved tile 3 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_3
📡 Downloading tile 4 at 47.8776770361181,-3.909947991759771 ...
✔ Saved tile 4 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_4
📡 Downloading tile 5 at 44.55378060775165,4.28389205759154 ...
✔ Saved tile 5 in /home/k2805/code/augustinlafond/project_urban_watch/raw_data/tile_5
📡 Downloading tile 6 at 45.69764498502806,5.8945655967385315 ...
✔ Saved tile 6 in /home/k2805/c

In [40]:
from urban_watch.ml_logic.data import load_data
from s2cloudless import S2PixelCloudDetector
X, meta = load_data()

In [41]:
X[0].shape

(300, 300, 5)

In [42]:
#prend 4 bandes: ,b2,b3,b4,b8
image_rgbnir = X[0][:,:,[0,1,2,3]].astype(float)

In [43]:
image_rgbnir_batch = image_rgbnir[np.newaxis, :, :, :]

In [44]:
dectetor = S2PixelCloudDetector(threshold=0.4, #sensi aux nuages
                                average_over=4,
                                dilation_size=2,
                                all_bands=False)

In [45]:
#calcul le masque de nuages
cloud_probs = dectetor.get_cloud_probability_maps(image_rgbnir_batch)
cloud_mask = cloud_probs > dectetor.threshold #si true = nuage

ValueError: Parameter `all_bands` is set to False, but images have 4 bands.